<a href="https://colab.research.google.com/github/qhungbui7/lenet-5/blob/main/lenet_5_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/

/content


In [3]:
!rm -r /content/lenet-5

In [4]:
!git clone https://github.com/qhungbui7/lenet-5.git

Cloning into 'lenet-5'...
remote: Enumerating objects: 1812, done.
remote: Counting objects: 100% (1812/1812), done.
remote: Compressing objects: 100% (1449/1449), done.
remote: Total 1812 (delta 320), reused 1785 (delta 307), pack-reused 0
Receiving objects: 100% (1812/1812), 2.43 MiB | 5.16 MiB/s, done.
Resolving deltas: 100% (320/320), done.


In [5]:
%%writefile /content/lenet-5/src/mnist.cc
#include "./mnist.h"

int ReverseInt(int i) {
  unsigned char ch1, ch2, ch3, ch4;
  ch1 = i & 255;
  ch2 = (i >> 8) & 255;
  ch3 = (i >> 16) & 255;
  ch4 = (i >> 24) & 255;
  return ((int)ch1 << 24) + ((int)ch2 << 16) + ((int)ch3 << 8) + ch4;
}

void MNIST::read_mnist_data(std::string filename, Matrix& data) {
  std::ifstream file(filename, std::ios::binary);
  if (file.is_open()) {
    int magic_number = 0;
    int number_of_images = 0;
    int n_rows = 0;
    int n_cols = 0;
    unsigned char label;
    file.read((char*)&magic_number, sizeof(magic_number));
    file.read((char*)&number_of_images, sizeof(number_of_images));
    file.read((char*)&n_rows, sizeof(n_rows));
    file.read((char*)&n_cols, sizeof(n_cols));
    magic_number = ReverseInt(magic_number);
    number_of_images = ReverseInt(number_of_images);
    n_rows = ReverseInt(n_rows);
    n_cols = ReverseInt(n_cols);

    data.resize(n_cols * n_rows, number_of_images);
    for (int i = 0; i < number_of_images; i++) {
      for (int r = 0; r < n_rows; r++) {
        for (int c = 0; c < n_cols; c++) {
          unsigned char image = 0;
          file.read((char*)&image, sizeof(image));
          data(r * n_cols + c, i) = (float)image;
        }
      }
    }
  }
}

void MNIST::read_mnist_label(std::string filename, Matrix& labels) {
  std::ifstream file(filename, std::ios::binary);
  if (file.is_open()) {
    int magic_number = 0;
    int number_of_images = 0;
    file.read((char*)&magic_number, sizeof(magic_number));
    file.read((char*)&number_of_images, sizeof(number_of_images));
    magic_number = ReverseInt(magic_number);
    number_of_images = ReverseInt(number_of_images);

    labels.resize(1, number_of_images);
    for (int i = 0; i < number_of_images; i++) {
      unsigned char label = 0;
      file.read((char*)&label, sizeof(label));
      labels(0, i) = (float)label;
    }
  }
}

void MNIST::read() {
  read_mnist_data(data_dir + "train-images.idx3-ubyte", train_data);
  read_mnist_data(data_dir + "t10k-images.idx3-ubyte", test_data);
  read_mnist_label(data_dir + "train-labels.idx1-ubyte", train_labels);
  read_mnist_label(data_dir + "t10k-labels.idx1-ubyte", test_labels);
}


Overwriting /content/lenet-5/src/mnist.cc


In [6]:
!mkdir ./lenet-5/data/
!mkdir ./lenet-5/data/mnist

In [7]:
!cp /content/drive/MyDrive/mnist/mnist.zip /content/mnist.zip

In [8]:
!unzip mnist.zip -d ./lenet-5/data/mnist

Archive:  mnist.zip
  inflating: ./lenet-5/data/mnist/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte  
  inflating: ./lenet-5/data/mnist/t10k-images.idx3-ubyte  
  inflating: ./lenet-5/data/mnist/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte  
  inflating: ./lenet-5/data/mnist/t10k-labels.idx1-ubyte  
  inflating: ./lenet-5/data/mnist/train-images-idx3-ubyte/train-images-idx3-ubyte  
  inflating: ./lenet-5/data/mnist/train-images.idx3-ubyte  
  inflating: ./lenet-5/data/mnist/train-labels-idx1-ubyte/train-labels-idx1-ubyte  
  inflating: ./lenet-5/data/mnist/train-labels.idx1-ubyte  


In [9]:
%cd ./lenet-5

/content/lenet-5


In [10]:
!ls ./data

mnist


In [11]:
%cd /content/lenet-5

/content/lenet-5


In [12]:
!mkdir /content/lenet-5/build
%cd /content/lenet-5/build
!cmake /content/lenet-5/
!make

/content/lenet-5/build
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done (0.6s)
-- Generating done (0.0s)
-- Build files have been written to: /content/lenet-5/build
[  6%] Building CXX object src/CMakeFiles/MiniDNNLib

In [13]:
!ls

CMakeCache.txt	CMakeFiles  cmake_install.cmake  demo  Makefile  src


In [14]:
!./demo

mnist train number: 60000
mnist test number: 10000
0-th batch, loss: 2.30217
50-th batch, loss: 2.3031
100-th batch, loss: 2.3004
150-th batch, loss: 2.30136
200-th batch, loss: 2.29618
250-th batch, loss: 2.2829
300-th batch, loss: 1.90509
350-th batch, loss: 0.603127
400-th batch, loss: 0.335416
450-th batch, loss: 0.23443

1-th epoch, test acc: 0.9146

0-th batch, loss: 0.354542
50-th batch, loss: 0.228444
100-th batch, loss: 0.131954
150-th batch, loss: 0.110524
200-th batch, loss: 0.143763
250-th batch, loss: 0.263087
300-th batch, loss: 0.105119
350-th batch, loss: 0.120148
400-th batch, loss: 0.123074
450-th batch, loss: 0.114843

2-th epoch, test acc: 0.9634

0-th batch, loss: 0.0962432
50-th batch, loss: 0.0980601
100-th batch, loss: 0.127678
150-th batch, loss: 0.13527
200-th batch, loss: 0.0956395
250-th batch, loss: 0.076831
300-th batch, loss: 0.116416
350-th batch, loss: 0.115276
400-th batch, loss: 0.0723094
450-th batch, loss: 0.0559415

3-th epoch, test acc: 0.9625

0-

In [15]:
# !nvcc HW2_P1.cu -o HW2_P1

In [16]:
# !./HW2_P1

In [17]:
# ! nvprof ./HW2_P1